In [1]:
#################################################
# Prepare training set using balance techniques #
#################################################

import numpy as np
import pandas as pd
from scipy import stats

from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN, SMOTETomek

import h2o
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [2]:
try:
    h2o.cluster().shutdown()
except AttributeError:
    pass
    
h2o.init(nthreads = -1,
         max_mem_size = "24G")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_171"; OpenJDK Runtime Environment (build 1.8.0_171-8u171-b11-0ubuntu0.16.04.1-b11); OpenJDK 64-Bit Server VM (build 25.171-b11, mixed mode)
  Starting server from /usr/local/lib/python2.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0wsRXu
  JVM stdout: /tmp/tmp0wsRXu/h2o_mourao_started_from_python.out
  JVM stderr: /tmp/tmp0wsRXu/h2o_mourao_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.4
H2O cluster version age:,2 months and 3 days
H2O cluster name:,H2O_from_python_mourao_go6wik
H2O cluster total nodes:,1
H2O cluster free memory:,21.33 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
# random seed
my_seed = 1980

In [4]:
# csv files' path
my_path = '../data/'

In [5]:
## load datasets
train = pd.read_csv(my_path + 'c2_e6_1_train.csv')
test = pd.read_csv(my_path + 'c2_e6_1_test.csv')

In [6]:
complete = train.columns.tolist()
remove = ['cd_pss', 'nm_mun_uor', 'sg_uf_uor', 'target', 'base_guess', 'label']
slim = [x for x in complete if x not in remove]

In [7]:
# separate train x and y
x = train[slim]
y = train['label']

In [8]:
# create h2o dataframe for test
test_h2o = h2o.H2OFrame(test[slim + ['label']])
test_h2o['label'] = test_h2o['label'].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
# samplers
samplers = ['RandomUnderSampler', 'ClusterCentroids', 'RandomOverSampler', 
            'SMOTE', 'ADASYN', 'SMOTEENN', 'SMOTETomek' ] 

In [10]:
def test_samplers(x, y, features, test_h2o, sampler, my_seed):
    s = eval(sampler + '(random_state=my_seed, ratio="all")')
    x_s, y_s = s.fit_sample(x, y)
    train_s = pd.concat([pd.Series(y_s, name='label'),
                       pd.DataFrame(x_s, columns=features)], axis=1)

    print(sampler)   
    print('Train: ' + str(len(train)) + ' records')
    print('Label ratio in training set: ' + str(100 * sum(train['label']) / len(train)) + '%')
    print
    print('Train resampled: ' + str(len(train_s)) + ' records')
    print('Label ratio in training set: ' + str(100 * sum(train_s['label']) / len(train_s)) + '%')
    print    
    
    score_sampler(train_s, test_h2o, features, sampler, my_seed)

In [11]:
def score_sampler(train, test_h2o, features, name, my_seed):
    train_h2o = h2o.H2OFrame(train)
    train_h2o['label'] = train_h2o['label'].asfactor()

    hyper_parameters = { 'alpha': [0, 0.1, 0.5, 0.7, 1.0] }

    lr_grid = H2OGridSearch(H2OGeneralizedLinearEstimator(family='binomial', 
                                                          seed=my_seed,
                                                          fold_assignment='Modulo',
                                                          lambda_search=True,
                                                          nfolds=5), 
                             grid_id=name,
                             hyper_params=hyper_parameters)

    lr_grid.train(x=features, 
                   y='label', 
                   training_frame=train_h2o)

    # Get the grid results, sorted by validation F1-Measure
    lr_gridperf1 = lr_grid.get_grid(sort_by='F1', decreasing=True)
    print(lr_gridperf1)

    # Grab the top LR model, chosen by validation F1
    best_lr1 = lr_gridperf1.models[0]

    # Now let's evaluate the model performance on a test set
    # so we get an honest estimate of top model performance
    best_lr_perf1 = best_lr1.model_performance(test_h2o)
    print(best_lr_perf1)

In [12]:
for sampler in samplers:
    test_samplers(x, y, slim, test_h2o, sampler, my_seed)

RandomUnderSampler
Train: 16636 records
Label ratio in training set: 4%

Train resampled: 1608 records
Label ratio in training set: 50%

Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Grid Build progress: |████████████████████████████████████████████████| 100%
     alpha                   model_ids                  f1
0    [1.0]  RandomUnderSampler_model_4  0.8251497005988024
1    [0.7]  RandomUnderSampler_model_3  0.8226779252110976
2    [0.1]  RandomUnderSampler_model_1  0.8220183486238531
3    [0.5]  RandomUnderSampler_model_2  0.8215158924205379
4    [0.0]  RandomUnderSampler_model_0   0.821014050091631


ModelMetricsBinomialGLM: glm
** Reported on test data. **

MSE: 0.108033881523
RMSE: 0.328685079556
LogLoss: 0.391499279147
Null degrees of freedom: 12476
Residual degrees of freedom: 12433
Null deviance: 17296.7947437
Residual deviance: 11117.1882961
AIC: 11205.1882961
AUC: 0.895044679476
Gini: 0.790089358952
Confusion Matrix (Act/Pred) for m

,0,1,Error,Rate
0,11680.0,217.0,0.0182,(217.0/11897.0)
1,305.0,275.0,0.5259,(305.0/580.0)
Total,11985.0,492.0,0.0418,(522.0/12477.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.8998219,0.5130597,27.0
max f2,0.6414935,0.5500686,95.0
max f0point5,0.9705271,0.5627828,9.0
max accuracy,0.9820827,0.9616094,6.0
max precision,0.9998040,0.8137931,0.0
max recall,0.0479551,1.0,379.0
max specificity,0.9998040,0.9977305,0.0
max absolute_mcc,0.9089627,0.4943758,25.0
max min_per_class_accuracy,0.4145523,0.8172414,171.0
max mean_per_class_accuracy,0.4403670,0.8202760,161.0


Gains/Lift Table: Avg response rate:  4,65 %



...

Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,0.8551244,0.5089974,39.0
max f2,0.6828793,0.5363224,87.0
max f0point5,0.9618040,0.5622010,12.0
max accuracy,0.9924623,0.9620101,3.0
max precision,0.9997085,0.8615385,0.0
max recall,0.0173759,1.0,389.0
max specificity,0.9997085,0.9984870,0.0
max absolute_mcc,0.8632549,0.4850511,37.0
max min_per_class_accuracy,0.4085607,0.8121375,173.0
max mean_per_class_accuracy,0.4246053,0.8144127,167.0


Gains/Lift Table: Avg response rate:  4,65 %



,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.0100184,0.9985853,18.7585241,18.7585241,0.872,0.872,0.1879310,0.1879310,1775.8524138,1775.8524138
,2,0.0200369,0.9668990,11.3583724,15.0584483,0.528,0.7,0.1137931,0.3017241,1035.8372414,1405.8448276
,3,0.0300553,0.9270481,9.4653103,13.1940690,0.44,0.6133333,0.0948276,0.3965517,846.5310345,1219.4068966
,4,0.0400737,0.8854255,6.7117655,11.5734931,0.312,0.538,0.0672414,0.4637931,571.1765517,1057.3493103
,5,0.0500120,0.8397409,5.8984705,10.4457643,0.2741935,0.4855769,0.0586207,0.5224138,489.8470523,944.5764257
,6,0.1000240,0.6713966,2.6545342,6.5501492,0.1233974,0.3044872,0.1327586,0.6551724,165.4534151,555.0149204
,7,0.1500361,0.5403850,1.4824022,4.8609002,0.0689103,0.2259615,0.0741379,0.7293103,48.2402188,386.0900199
,8,0.2000481,0.4349507,1.3100298,3.9731826,0.0608974,0.1846955,0.0655172,0.7948276,31.0029841,297.3182609
,9,0.2999920,0.2955629,0.8625529,2.9368601,0.0400962,0.1365215,0.0862069,0.8810345,-13.7447114,193.6860070
,10,0.4000160,0.2100107,0.3619819,2.2930116,0.0168269,0.1065919,0.0362069,0.9172414,-63.8018070,129.3011559


/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 16:48:57.214553. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
train.columns

Index([u'cd_pss', u'nm_mun_uor', u'sg_uf_uor', u'label', u'target',
       u'base_guess', u'v1', u'v2', u'v3', u'v4', u'v5', u'v6', u'v7', u'v8',
       u'v9', u'v10', u'v11', u'v12', u'v13', u'v14', u'v15', u'v16', u'v17',
       u'v18', u'v19', u'v20', u'v21', u'v22', u'v23', u'v24', u'v25', u'v26',
       u'v27', u'v28', u'v29', u'v30', u'v31', u'v32', u'v33', u'v34', u'v35',
       u'v36', u'v37', u'v38', u'v39', u'v40', u'v41', u'v42', u'v43', u'v44',
       u'v45', u'v46', u'v47', u'v48', u'v49', u'v50'],
      dtype='object')

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 17:00:20.367410. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [14]:
# save best sampler
remove = ['cd_pss', 'nm_mun_uor', 'sg_uf_uor', 'target', 'label']
features = [x for x in train.columns if x not in remove]

s = RandomOverSampler(random_state=my_seed)
x_s, y_s = s.fit_sample(train[features], train['label'])
train_s = pd.concat([pd.Series(y_s, name='label'),
                   pd.DataFrame(x_s, columns=features)], axis=1)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 17:00:20.377420. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [15]:
train_s.to_csv(my_path + 'c2_e6_2_train.csv')

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 17:00:20.441608. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [16]:
# create libsvm files inspired in https://github.com/zygmuntz/phraug

def create_libsvm(df, features, out):

    with open(out, 'w') as w:
        for index, row in df.iterrows():
            new_row = str(int(row['label']))
            for i, col in enumerate(features):
                new_row += ' %s:%s' % (i + 1, str(row[col]))
            w.write(new_row + '\n')

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 17:00:21.528261. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [17]:
create_libsvm(train_s, slim, my_path + 'c2_e6_2_train.svm')
create_libsvm(test, slim, my_path + 'c2_e6_2_test.svm')

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2018-05-12 17:00:21.535195. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
